In [ ]:
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

import os

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Keras specific
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, GaussianNoise, Dropout, BatchNormalization
from keras import initializers
from keras.optimizers import SGD, Nadam, RMSprop
from keras.constraints import maxnorm
from keras.regularizers import l2
from keras.layers.advanced_activations import LeakyReLU, PReLU
#from keras.wrappers.scikit_learn import KerasRegressor
#from sklearn import preprocessing 
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.pipeline import Pipeline
#from scipy.stats.stats import pearsonr
#from scikit_learn.model_selection import train_test_split
#from scikit_learn.metrics import mean_absolute_error
import numpy as np
import tensorflow as tf
import keras.backend as kb
import keras.backend as K
import os
import threading
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


#df = pd.read_csv('traindats.csv')
dataset = pd.read_csv('New_data.csv', usecols= ['TotalArea','TailArea','Classification'])

print(dataset) 

X = dataset.drop(['Classification'], axis=1)
Y = dataset['Classification']

Neutrons = 0
Gammas   = 0
for i in range(len(Y)):
    if Y[i]=='Neutron':
        Neutrons = Neutrons + 1
    else: 
        Gammas    = Gammas   + 1
        
print(Neutrons,Gammas) 

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(["Neutron", "Gamma"])
Y = le.transform(dataset['Classification'])
print(Y.T)

Neutron = 0
Gamma   = 0

for i in range(len(Y)):
    if Y[i]==1:
        Neutron = Neutron+1
    else:
        Gamma   = Gamma  +1
   
print(Neutron,Gamma)

print(f'X shape: {X.shape} | y shape: {Y.shape} ')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=40)

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler = MinMaxScaler()
#scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)
#print(X_train)


from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold


model = Sequential()
model.add(Dense(64, input_dim=2, activation='relu', kernel_initializer='uniform'))
model.add(Dense(32,              activation='relu', kernel_initializer='uniform'))
model.add(Dense(16,              activation='relu', kernel_initializer='uniform'))
model.add(Dense(1,               activation='sigmoid'))
#model.add(Dense(1,               activation='softmax'))
# compile model
opt = SGD(lr=0.01, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
history = model.fit(X_train, Y_train, validation_split=0.1, epochs=200, verbose=1)
# evaluate the model
_, train_acc = model.evaluate(X_train, Y_train, verbose=0)
_, test_acc = model.evaluate(X_test, Y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot loss during training
plt.figure(1)
plt.title('Loss')
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()
# plot accuracy during training
plt.figure(2)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

## save your model 
##serialize model to JSON 
model_json = model.to_json()
with open("DNNmodelMOD.json","w") as json_file:
    json_file.write(model_json)
##serialize weights to HDF5
model.save_weights("DNNmodelMOD.h5")
print("saved model to disk")

print(history.history.keys())
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc =history.history['accuracy']
val_acc = history.history['val_accuracy']

np.save('training_lossDNN.npy',train_loss)
np.save('validation_lossDNN.npy', val_loss)

#with plt.rc_context({'axes.edgecolor':'black', 'xtick.color':'black', 'ytick.color':'black', 'figure.facecolor':'white'}):
plt.figure()
plt.plot(train_loss,'r',linewidth=3)
plt.plot(val_loss,'b',linewidth=3,linestyle='dashed')
plt.xlabel('Number of epochs',fontweight='bold',fontsize=20.0)
plt.ylabel('Losses',fontweight='bold',fontsize=20.0)
plt.legend(["Training loss", "Validation loss"], loc ="upper right")
  
plt.tick_params(axis="x", direction="in")
plt.tick_params(axis="y", direction="in")
    
plt.savefig('DNN_losses.png', bbox_inches = 'tight')
plt.show()

score = model.evaluate(X_test, Y_test, verbose=1)
print('Accuracy on test data from loaded data: {} \n Error on test data: {}'.format(score[1], 1 - score[1]))
print(score)

from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

font = {'family':'normal','weight':'bold','size': 18}
plt.rc('font', **font)

y_predicted = model.predict(X_test) > 0.5

# Calculate the confusion matrix
#
conf_matrix = confusion_matrix(y_true=Y_test, y_pred=y_predicted)
#
# Print the confusion matrix using Matplotlib
#
fig, ax = plt.subplots(figsize=(9, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Greens, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
ax.xaxis.set_ticklabels(['Gammas', 'Neutrons']); 
ax.yaxis.set_ticklabels(['Gammas', 'Neutrons']);
plt.show()

#plt.savefig('ConfusionMatrixtest.png')

with plt.rc_context({'axes.edgecolor':'black', 'xtick.color':'black', 'ytick.color':'black', 'figure.facecolor':'white'}):
    from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report 

    fig, ax = plt.subplots(figsize = (9, 7.5))
    cm = confusion_matrix(Y_test,y_predicted)

    disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ["Gamma", "Neutron"])
    #disp.plot(cmap = plt.cm.Blues, ax = ax)
    disp.plot(cmap = plt.cm.Blues, ax = ax, values_format='')
    ax.set_title("Confusion Matrix",fontsize=22,fontweight='bold')
    plt.xlabel('Predicted Label', color ='black',fontsize=22,fontweight='bold')
    plt.ylabel('True Label', color ='black',fontsize=22,fontweight='bold')
    
    plt.tick_params(axis="x", direction="in")
    plt.tick_params(axis="y", direction="in")
    plt.savefig('ConfusionMatrixtest.png', bbox_inches = 'tight')   
    plt.show()

# classification report for precision, recall f1-score and accuracy
    matrix = classification_report(Y_test,y_predicted,labels=[1,0])
    print('Classification report : \n',matrix)
    
    
#####################################                   ID 3               ################################################################    
    
datasetGGAG = pd.read_csv('GGAGTEST.csv', usecols= ['TotalArea','TailArea','Classification'])

#print(dataset) 

X_test = datasetGGAG.drop(['Classification'], axis=1)
Y_test = datasetGGAG['Classification']

#print(X_test.shape)
#print(Y_test.shape)

Neutrons = 0
Gammas   = 0
for i in range(len(Y_test)):
    if Y_test[i]=='Neutron':
        Neutrons = Neutrons + 1
    else: 
        Gammas    = Gammas   + 1
        
print(Neutrons,Gammas) 

from sklearn import preprocessing

Y_test = le.transform(datasetGGAG['Classification'])
#print(Y_test)

print(f'X shape: {X_test.shape} | y shape: {Y_test.shape} ')

from sklearn.preprocessing import MinMaxScaler

X_test  = scaler.transform(X_test)

#print(X_test)

y_predicted = model.predict(X_test) > 0.5

with plt.rc_context({'axes.edgecolor':'black', 'xtick.color':'black', 'ytick.color':'black', 'figure.facecolor':'white'}):
    from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report 

    fig, ax = plt.subplots(figsize = (9, 7.5))
    cm = confusion_matrix(Y_test,y_predicted)

    disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ["Gamma", "Neutron"])
    #disp.plot(cmap = plt.cm.Blues, ax = ax)
    disp.plot(cmap = plt.cm.Blues, ax = ax, values_format='')
    ax.set_title("Confusion Matrix",fontsize=22,fontweight='bold')
    plt.xlabel('Predicted Label', color ='black',fontsize=22,fontweight='bold')
    plt.ylabel('True Label', color ='black',fontsize=22,fontweight='bold')
    
    plt.tick_params(axis="x", direction="in")
    plt.tick_params(axis="y", direction="in")
    plt.savefig('ConfusionMatrixGGAG.png', bbox_inches = 'tight')   
    plt.show()
    
    matrix = classification_report(Y_test,y_predicted,labels=[1,0])
    print('Classification report : \n',matrix)
    

#############################################            ID 2                      ########################################################


datasetGGAG = pd.read_csv('testdataBC501.csv', usecols= ['TotalArea','TailArea','Classification'])

#print(dataset) 

X_test = datasetGGAG.drop(['Classification'], axis=1)
Y_test = datasetGGAG['Classification']

#print(X_test.shape)
#print(Y_test.shape)

Neutrons = 0
Gammas   = 0
for i in range(len(Y_test)):
    if Y_test[i]=='Neutron':
        Neutrons = Neutrons + 1
    else: 
        Gammas    = Gammas   + 1
        
print(Neutrons,Gammas) 

from sklearn import preprocessing

Y_test = le.transform(datasetGGAG['Classification'])
#print(Y_test)

print(f'X shape: {X_test.shape} | y shape: {Y_test.shape} ')

from sklearn.preprocessing import MinMaxScaler

X_test  = scaler.transform(X_test)

#print(X_test)

y_predicted = model.predict(X_test) > 0.5

with plt.rc_context({'axes.edgecolor':'black', 'xtick.color':'black', 'ytick.color':'black', 'figure.facecolor':'white'}):
    from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report 

    fig, ax = plt.subplots(figsize = (9, 7.5))
    cm = confusion_matrix(Y_test,y_predicted)

    disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ["Gamma", "Neutron"])
    #disp.plot(cmap = plt.cm.Blues, ax = ax)
    disp.plot(cmap = plt.cm.Blues, ax = ax, values_format='')
    ax.set_title("Confusion Matrix",fontsize=22,fontweight='bold')
    plt.xlabel('Predicted Label', color ='black',fontsize=22,fontweight='bold')
    plt.ylabel('True Label', color ='black',fontsize=22,fontweight='bold')
    
    plt.tick_params(axis="x", direction="in")
    plt.tick_params(axis="y", direction="in")
    plt.savefig('ConfusionMatrixBC501BARC.png', bbox_inches = 'tight')   
    plt.show()
    
    matrix = classification_report(Y_test,y_predicted,labels=[1,0])
    print('Classification report : \n',matrix)    
    

####################################################    ID1       ####################################################################    
    
datasetGGAG = pd.read_csv('testdata1.csv', usecols= ['TotalArea','TailArea','Classification'])

#print(dataset) 

X_test = datasetGGAG.drop(['Classification'], axis=1)
Y_test = datasetGGAG['Classification']

#print(X_test.shape)
#print(Y_test)

Neutrons = 0
Gammas   = 0
for i in range(len(Y_test)):
    if Y_test[i]=='Neutron':
        Neutrons = Neutrons + 1
    else: 
        Gammas    = Gammas   + 1
        
print(Neutrons,Gammas) 

from sklearn import preprocessing

Y_test = le.transform(datasetGGAG['Classification'])
#print(Y_test)

print(f'X shape: {X_test.shape} | y shape: {Y_test.shape} ')

from sklearn.preprocessing import MinMaxScaler

X_test  = scaler.transform(X_test)

#print(X_test)

y_predicted = model.predict(X_test) > 0.5

with plt.rc_context({'axes.edgecolor':'black', 'xtick.color':'black', 'ytick.color':'black', 'figure.facecolor':'white'}):
    from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report 

    fig, ax = plt.subplots(figsize = (9, 7.5))
    cm = confusion_matrix(Y_test,y_predicted)

    disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ["Gamma", "Neutron"])
    #disp.plot(cmap = plt.cm.Blues, ax = ax)
    disp.plot(cmap = plt.cm.Blues, ax = ax, values_format='')
    ax.set_title("Confusion Matrix",fontsize=22,fontweight='bold')
    plt.xlabel('Predicted Label', color ='black',fontsize=22,fontweight='bold')
    plt.ylabel('True Label', color ='black',fontsize=22,fontweight='bold')
    
    plt.tick_params(axis="x", direction="in")
    plt.tick_params(axis="y", direction="in")
    titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
#    plt.savefig('ConfusionMatrixBC501IITK.png', bbox_inches = 'tight')   
    plt.show()
    
    matrix = classification_report(Y_test,y_predicted,labels=[1,0])
    print('Classification report : \n',matrix)    